In [1]:
from openvino.inference_engine import IECore

In [2]:
import openvino,cv2
import os

os.sys.path.append("../")

In [3]:
import pkg_resources
import logging

In [4]:
openvino.inference_engine.__version__

'2.1.42025'

In [23]:
ie.get_metric(metric_name="OPTIMIZATION_CAPABILITIES", device_name="HETERO")

RuntimeError: Unsupported Plugin metric: OPTIMIZATION_CAPABILITIES

In [4]:
ie = IECore()
model_path = "/home/prateek/prateek_space/intel_model_dump/intel/face-detection-adas-0001/FP32/face-detection-adas-0001.xml"
net = ie.read_network(model=model_path, weights=model_path.replace(".xml",".bin"))
exec_net = ie.load_network(net, "GPU", num_requests=4)

In [10]:
ie.set_config({'DYN_BATCH_ENABLED': 'YES'},'GPU')

In [13]:
list(net.inputs.keys())

['data']

In [35]:
net.outputs

{'detection_out': <openvino.inference_engine.ie_api.DataPtr at 0x7fe9568b12d0>}

In [32]:
exec_net.inputs

{'data': <openvino.inference_engine.ie_api.DataPtr at 0x7fe9568b15f0>}

In [33]:
exec_net.outputs

{'detection_out': <openvino.inference_engine.ie_api.CDataPtr at 0x7fe9568b1c90>}

In [14]:
im = cv2.imread("/home/prateek/prateek_space/helmet_n_vest/2019_12_26_cleaned_helmet_vest_pallavi/camera_2/train_frames/0000001.jpeg")
im1 =  cv2.resize(im,(672,384))
im2 = im1.reshape(exec_net.inputs['data'].shape)

In [15]:
callback = lambda status, py_data: print("Request with id {} finished with status {}".format(py_data, status))


In [16]:
for id, req in enumerate(exec_net.requests):
    req.set_completion_callback(py_callback=callback, py_data=id)

In [17]:
for i in range(4):
    exec_net.requests[i].async_infer({"data":im2})

Request with id 0 finished with status 0
Request with id 1 finished with status 0
Request with id 2 finished with status 0
Request with id 3 finished with status 0


In [11]:
for i in range(4):
    res = exec_net.requests[i].outputs


In [13]:
exec_net.inputs

{'data': <openvino.inference_engine.ie_api.DataPtr at 0x7f718b65dc50>}

In [15]:
exec_net.inputs['data'].shape

[1, 3, 384, 672]

In [5]:
ie.get_metric(metric_name="SUPPORTED_METRICS", device_name="CPU",)


['AVAILABLE_DEVICES',
 'SUPPORTED_METRICS',
 'FULL_DEVICE_NAME',
 'OPTIMIZATION_CAPABILITIES',
 'SUPPORTED_CONFIG_KEYS',
 'RANGE_FOR_ASYNC_INFER_REQUESTS',
 'RANGE_FOR_STREAMS']

In [11]:
import support_utility_openvino

In [12]:
support_utility_openvino.async_infer.preprocess_frame(im)

TypeError: preprocess_frame() missing 1 required positional argument: 'frame'

In [18]:
exec_net.infer({'data':im2})

{'detection_out': array([[[[0.        , 1.        , 0.07635968, ..., 0.8205075 ,
           0.7707091 , 0.87830114],
          [0.        , 1.        , 0.07393593, ..., 0.6182586 ,
           0.7645484 , 0.6855664 ],
          [0.        , 1.        , 0.07346141, ..., 0.540742  ,
           0.2861867 , 0.5902806 ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]]]], dtype=float32)}

In [16]:
exec_net.export("test/test_net")

RuntimeError: [NOT_IMPLEMENTED] 

In [ ]:
for id, req in enumerate(exec_net.requests):
    req.set_completion_callback(py_callback=callback, py_data=id)
for req in exec_net.requests:
    req.async_infer({"data": img})